<a href="https://colab.research.google.com/github/myy04/Real-Fake-Image-Classifier/blob/main/resnet18_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import AutoModel, AutoProcessor
import os
import torchvision
import torchvision.transforms as transforms
from torch import Tensor

import warnings
warnings.filterwarnings("ignore")

In [20]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images/versions/3


In [21]:
class ImageDataset(Dataset):
    def __init__(self, path):

        real_images_directory = os.path.join(path, 'REAL')
        fake_images_directory = os.path.join(path, 'FAKE')

        real_images = []
        fake_images = []

        for file in os.listdir(real_images_directory):
            real_images.append(os.path.join(real_images_directory, file))

        for file in os.listdir(fake_images_directory):
            fake_images.append(os.path.join(fake_images_directory, file))

        self.images = []
        for i in range(min(len(real_images), len(fake_images))):
            self.images.append((real_images[i], torch.tensor([1, 0], dtype = torch.float32)))
            self.images.append((fake_images[i], torch.tensor([0, 1], dtype = torch.float32)))

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Resize to a standard size for ResNet
            transforms.ToTensor(),          # Convert PIL Image to PyTorch Tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx][0]
        image = Image.open(image_path)

        image = self.transform(image)

        return image, self.images[idx][1]


BATCHSIZE = 128
NEPOCH = 20

train_dataset = ImageDataset(os.path.join(path, 'train'))
eval_dataset = ImageDataset(os.path.join(path, 'test'))

train_dataloader = DataLoader(train_dataset, batch_size = BATCHSIZE, shuffle = True)
eval_dataloader = DataLoader(eval_dataset, batch_size = BATCHSIZE, shuffle = True)


In [22]:
resnet_model = torchvision.models.resnet18(pretrained = False)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet_model = resnet_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_model.parameters(), weight_decay=1e-3)

In [23]:
def train_step(input: Tensor, label: Tensor):
  resnet_model.train()

  input = input.to(device)
  label = label.to(device)

  output = resnet_model(input)
  loss = criterion(output, label)

  optimizer.zero_grad()
  loss.backward()

  torch.nn.utils.clip_grad_norm_(resnet_model.parameters(), max_norm=1.0)
  optimizer.step()

  return loss, output

In [24]:
@torch.no_grad()
def test_step(input: Tensor, label: Tensor):

    resnet_model.eval()

    input = input.to(device)
    label = label.to(device)

    output = resnet_model(input)

    loss = criterion(output, label)

    return loss, output

In [25]:
def train_epoch(dataloader):
    correct = 0
    samples = 0

    for i, (input, label) in enumerate(dataloader):
        loss, output = train_step(input, label)

        _, predicted = torch.max(output.data, 1)

        samples += label.size(0)
        # The label tensor has shape (batch_size, 2) - we need to compare predicted to the index with the maximum value in the label tensor
        correct += (predicted == torch.argmax(label, dim=1).to(device)).sum().item()

        if i % 100 == 0:
            print(f"Loss [{i}/{len(dataloader)}]: {loss.item()}")

    print(f"Train Accuracy: {correct / samples * 100.00}%")

In [26]:
def test_epoch(dataloader):
    correct = 0
    samples = 0

    for i, (input, label) in enumerate(dataloader):
        loss, output = test_step(input, label)

        _, predicted = torch.max(output.data, 1)

        samples += label.size(0)
        correct += (predicted == torch.argmax(label, dim=1).to(device)).sum().item()

        if i % 100 == 0:
            print(f"Loss [{i}/{len(dataloader)}]: {loss.item()}")

    print(f"Test Accuracy: {correct / samples * 100.00}%")

In [27]:
for epoch in range(NEPOCH):
    train_epoch(train_dataloader)

    test_epoch(eval_dataloader)

torch.save(resnet_model.state_dict(), 'resnet_model_parameters.pth')

Loss [0/782]: 0.7163615226745605
Loss [100/782]: 0.6752097606658936
Loss [200/782]: 0.6483266353607178
Loss [300/782]: 0.6146159768104553
Loss [400/782]: 0.6170250177383423
Loss [500/782]: 0.550330638885498
Loss [600/782]: 0.5632714033126831
Loss [700/782]: 0.5873779058456421
Train Accuracy: 70.282%
Loss [0/157]: 0.5433982610702515
Loss [100/157]: 0.5646390914916992
Test Accuracy: 75.84%
Loss [0/782]: 0.5585246086120605
Loss [100/782]: 0.5091971158981323
Loss [200/782]: 0.5622423887252808
Loss [300/782]: 0.491061270236969
Loss [400/782]: 0.4896157681941986
Loss [500/782]: 0.4443996846675873
Loss [600/782]: 0.4582618176937103
Loss [700/782]: 0.4932329058647156
Train Accuracy: 77.73%
Loss [0/157]: 0.4603258967399597
Loss [100/157]: 0.49428486824035645
Test Accuracy: 79.66%
Loss [0/782]: 0.4920300841331482
Loss [100/782]: 0.4918546676635742
Loss [200/782]: 0.4115431010723114
Loss [300/782]: 0.4064381718635559
Loss [400/782]: 0.48792868852615356
Loss [500/782]: 0.4037911295890808
Loss [600